In [1]:
from IPython.display import clear_output
import pandas as pd
from pandas import Series
import numpy as np
import re
import csv
from gsheets import Sheets
from datetime import datetime

In [2]:
url = 'https://docs.google.com/spreadsheets/d/1LRbios7yQRo3aqCh0Es2Wiae_dicg_OtL-_yqP-Tb8I/edit#gid=1718343431'

## Create resusable lists for reference in data display or collection

In [30]:
# lists used in the loading of the AAT Metadata
terms_header_list = ["AACR2_FLAG", "DISPLAY_DATE", "DISPLAY_NAME", "DISPLAY_ORDER", "END_DATE", "HISTORIC_FLAG", "OTHER_FLAGS", "PREFERRED", "START_DATE", "SUBJECT_ID", "TERM", "TERM_ID", "VERNACULAR"]
terms_dtype = {'TERM_ID': str, 'SUBJECT_ID' : str}
lang_rels_header_list = ["LANGUAGE_CODE", "PREFERRED", "SUBJECT_ID", "TERM_ID", "QUALIFIER", "TERM_TYPE", "PART_OF_SPEECH", "LANG_STAT"]
lang_rel_dtype = {'TERM_ID': str, 'SUBJECT_ID' : str, 'LANGUAGE_CODE': str}
narrow_list = ['identifier', 'media', 'subject', 'title', 'subjects', 'SUBJECT_ID_x', 'TERM_ID', 'join_concept', 'BoxId', 'color']

## Functions

In [4]:
# load metadata

def load_metadata() -> pd.DataFrame:
    """load the metadata that will be used
    files were downloaded from: http://aatdownloads.getty.edu/
    """
    terms = read_aat_terms('./AAT_Files/TERM.out', terms_header_list, terms_dtype)
    lang_rels = read_aat_terms('./AAT_Files/LANGUAGE_RELS.out', lang_rels_header_list, lang_rel_dtype)
    concepts = (pd.merge(terms, lang_rels, how = 'inner', on='TERM_ID'))
    concepts['full_concept'] = (concepts.TERM + ' (' + concepts.QUALIFIER + ')')
    concepts['use_concept'] = (concepts.full_concept.fillna(concepts.TERM))
    concepts['join_concept'] = (concepts.use_concept.str.lower())

    return concepts

In [5]:
def read_aat_terms(file_path: str, names: list, data_types: dict) -> pd.DataFrame:
    data = pd.read_csv(file_path
                       , sep = '\t'
                       , warn_bad_lines=True
                       , error_bad_lines=False
                       , names=names
                       , dtype=data_types)
    return data

In [6]:
# get box data by box number (as an integer)

def return_box(box_num: int) -> pd.DataFrame:
    box_id = f'Box {str(box_num)}'
    df = s.find(box_id).to_frame()
    df = format_column_names(df)
    return df

In [7]:
# get column names in the correct format during the retrieval of box data

def format_column_names(df: pd.DataFrame) -> pd.DataFrame:
    column_names = list(map(lambda x: x.strip().lower().replace("dcterms:", ""), df.columns))
    df = df.set_axis(column_names, axis=1, inplace=False)
    return df

In [8]:
def split_df(df: pd.DataFrame) -> pd.DataFrame:
    """given a dataframe with multiple concepts specified, splits them and returns them 
    as one row per concept
    """
    split_df = df['subject'].str.split(',', expand=True).apply(Series, 1).stack()
    split_df.index = split_df.index.droplevel(-1)
    split_df.name = 'subjects'

    x = df.join(split_df)
    x['subjects'] = x['subjects'].str.strip()
    x['join_concept'] = x['subjects'].str.lower()
    
    return x

In [9]:
def terminate() -> str:
    """
    check to see if we should stop or continue the loop
    
    """
    
    terminate = input("Press 'X' key to terminate the program")
    if terminate == 'X':
        print('stopping')
    return terminate

In [10]:
def combine_boxes(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    """assuming df1 and df2 are dataframes with same schema, combines them and returns them"""
    df_concat = pd.concat([df1, df2])
    return df_concat

In [11]:
def add_to_not_found(df: pd.DataFrame):
    """write to a csv the records that could not be matched"""
    
    file_name = 'csv_files/unmatched_subjects_v2.csv'
    
    try:
        old_df = pd.read_csv(file_name, header=0, index_col=False)
        combined_df = pd.concat([old_df, df])
    except(FileNotFoundError):
        combined_df = df
    
    combined_df.to_csv(file_name, index=False, quoting=csv.QUOTE_ALL)

## Load the spreadsheet information

In [12]:
# if HttpAccessTokenRefreshError, close everything and start over
sheets = Sheets.from_files('~/client_secrets.json', '~/storage.json')
s = sheets.get(url)

In [354]:
# get the list of boxes containing metadata

box_list = []
box_list_sheets = s.sheets.titles()

for b in box_list_sheets:
    if 'Box' in b:
        box_list.append(int(b[4:]))

## Load the Metadata from Getty AAT

Individual AAT Terms can be seen here:
https://www.getty.edu/research/tools/vocabularies/aat/

We use the .Out files available below for consuming the data - these are batch released so terms in the .Out files may note be availalbe on the AAT search (link above) or vice-versa:
http://aatdownloads.getty.edu/

Note that when we load files, we join the LANGUAGE_RELS.out and TERM.out files and combine the term and and the qualifier to create the "subjects" and "join_concepts"

The join_concept ensures the case for the term is all lower case

We also create a "full_concept" but doesn't handle the NULL case well and shouldn't be used for analysis

In [14]:
concepts = load_metadata()

/Users/ethan.fenichel/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
# review the metadata for concepts containing a comma and return those as a list
concepts_drop = concepts[concepts['use_concept'].notna()]
concepts_search = concepts_drop[concepts_drop['use_concept'].str.contains(',')]
concepts_comma = concepts_search['use_concept'].to_list()

In [16]:
concepts[concepts['join_concept'].notna()]

,AACR2_FLAG,DISPLAY_DATE,DISPLAY_NAME,DISPLAY_ORDER,END_DATE,HISTORIC_FLAG,OTHER_FLAGS,PREFERRED_x,START_DATE,SUBJECT_ID_x,...,LANGUAGE_CODE,PREFERRED_y,SUBJECT_ID_y,QUALIFIER,TERM_TYPE,PART_OF_SPEECH,LANG_STAT,full_concept,use_concept,join_concept
0,NaN,NaN,NaN,3.0,NaN,C,NaN,V,NaN,300022903,...,70051,N,300022903,NaN,UF,U,U,NaN,"knives, gauge","knives, gauge"
1,NaN,NaN,NaN,1.0,NaN,C,NaN,P,NaN,300022904,...,70051,P,300022904,NaN,D,PN,U,NaN,hand knives,hand knives
2,NaN,NaN,NaN,2.0,NaN,C,NaN,V,NaN,300022904,...,70051,N,300022904,NaN,AD,SN,U,NaN,hand knife,hand knife
3,NaN,NaN,NaN,3.0,NaN,C,NaN,V,NaN,300022904,...,70051,N,300022904,NaN,UF,U,U,NaN,"knives, hand","knives, hand"
4,NaN,NaN,NaN,1.0,NaN,C,NaN,P,NaN,300022905,...,70051,P,300022905,NaN,D,PN,U,NaN,hawkbill knives,hawkbill knives
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483188,NaN,NaN,NaN,1.0,NaN,C,NaN,P,NaN,300430882,...,70051,P,300430882,NaN,D,U,U,NaN,prophylactics,prophylactics
483189,NaN,NaN,NaN,1.0,NaN,C,NaN,P,NaN,300434048,...,70051,P,300434048,NaN,D,PN,U,NaN,styptic pencils,styptic pencils
483190,NaN,NaN,NaN,1.0,NaN,C,NaN,P,NaN,300434776,...,70051,P,300434776,NaN,D,PN,U,NaN,tissues,tissues
483191,NaN,NaN,NaN,1.0,NaN,C,NaN,P,NaN,300431243,...,70051,P,300431243,NaN,D,PN,U,NaN,towelettes,towelettes


### Consider comma cases

given that we'll parse the created Arquin metadata by comma, we need to handle cases where the subject term, aka concept, contains an internal comma (e.g. "knives, gauge"). These will need to be parsed carfully for two reasons:

1.) The comma could falsely parse the concept

2.) The individual components of the concept could be valid AAT terms, too... eg. "knives, gauge" Both "knives" and "gauge" are valid AAT terms while so is "knives, gauge". Knowing what the metata analyst intended in this case is highly difficult to determine. 

In [17]:
# display a sample of concepts_comma
concepts_comma[0:10]

['knives, gauge',
 'knives, hand',
 'knives, hawkbill',
 'mill, feed',
 'mills, feed',
 'reports, congressional committee',
 'bills, House of Commons',
 'bills, House of Lords',
 'charts, bathymetric',
 'looms, warp-weighted']

In [18]:
# identify the risk of concepts that have multiple commas in them

multi_commas_list = []
for concept in concepts_comma:
    x = concept.count(',')
    if x > 1:
        multi_commas_list.append(concept)

print(f'There are {len(multi_commas_list)} concepts that have more than 1 comma')

There are 211 concepts that have more than 1 comma


In [19]:
# show a sample of the concepts with more than 1 comma
multi_commas_list[34:44]

['Zhou, Northern (culture, style, and period)',
 'Liang (culture, style, period)',
 'Animal, Early, Style',
 'Animal, Late, Style',
 'Ripe, Animal, Late',
 'Animal, Middle, Style',
 'Eastern Zambian, Zimbabwe, and Western Mozambique styles',
 'Jin (Six Dynasties, culture, style, and period)',
 'Chin (Six Dynasties, culture, style, and period)',
 'Later Liang (Five Dynasties, Chinese style, culture, period)']

## Begin parsing the subjects in the boxes

This process will parse the subjects and then evaluate them.

1. parse the subjects by comma (really a comma+space, ', ')
2. for all the subjects associated with a given media id, validate that they each match an AAT join_concept
3. If yes, move on to the next media id
4. If a parsed join_concept fails to match an AAT Term, identify the order of that term in the series of subjects (e.g. if there are 5 parsed terms in the set of 'subject', and 1, 2, and 5 match an AAT term). Then combine the nearest term with a subject (3 and 4 from the example) and see if that term matches AAT.
5. If yes (in 4), remove subjects 3 and 4 and replace with the combined 3 & 4. 
6. If no (in 4), log the unmatched terms for further analysis

## Develop the flow for this to work in an interation

In [345]:
# box_list = [7]

In [370]:
# load all the boxes and split them
for i, box_number in enumerate(box_list):
    df = return_box(box_number)
    clear_output(wait=True)
    print(f"working on Box {box_number}")
    
    try:
        df = split_df(df)
    except:
        print(f'failed to split df for box number {box_number}')

    if i+1 == 1:
        df_combined = df
    else:
        df_combined = combine_boxes(df_combined, df)
 
clear_output(wait=True)    

df = df_combined
df = df.drop(df[df['subjects'] == ''].index)
print(f'complete, imported {i+1} boxes')

complete, imported 69 boxes


In [371]:
# join the df with the concepts
df = pd.merge(df, concepts, how="left", on='join_concept')

In [372]:
# get the boxid
df['boxid_temp'] = df['media'].str.extract(r"(_0\d+_)")
df = df[df['media'].notna()]
df['boxid_temp'] = df['boxid_temp'].str.replace("_","")
df['boxid_temp'] = pd.to_numeric(df['boxid_temp'], errors='coerce')
df = df.dropna(subset=['boxid_temp'])
df['boxid_temp'] = df['boxid_temp'].astype('int')
df['boxid_temp'] = df['boxid_temp'].astype('str')
df['BoxId'] = 'Box '+df['boxid_temp']
df.drop('boxid_temp', inplace=True, axis=1)

## Add in Tab Color

The colors of each tab are identified in 'get_spreadsheets_color.ipynb'. That script creates a CSV file and that is read in and added to the df_combined object

Read in the color match file created here:
http://localhost:8888/notebooks/get_spreadsheets.ipynb

use that to create the list of titles based on color of the tab

In [373]:
color_match_df = pd.read_csv('csv_files/color_match', sep='\t')

In [374]:
df = pd.merge(df, color_match_df, on='BoxId')

In [375]:
# df = df[df['media'] == 'B-1_Arquin_007_0257.jpg']
# df[narrow_list]

In [376]:
# get all the unique media IDs where media is not NA 
df_medias = df.media.dropna().unique()

In [377]:
df_medias_len = len(df_medias)

for i, search_media in enumerate(df_medias):
    clear_output(wait=True) 
    print(f'{i} of {df_medias_len}: search_media = {search_media}')
#     x = input('step 1: press any key to continue... ')
    
    try_match_neg = None
    try_match_pos = None
    cancel_repeat = False
    concepts_found_pos = []
    concepts_found_neg = []
    
    df_subjects = df[df['media'] == search_media].join_concept.to_list()
    df_media_no_match = df[df['media'] == search_media].loc[df['SUBJECT_ID_x'].isna()]
#     print(f'{i}: search_media = {search_media}, df_media_no_match = {df_media_no_match}')
#     x = input('press any key to continue... ')
    
    if len(df_media_no_match) > 0:
#         print(f'{i}: search_media = {search_media}, passed len(df_media_no_match) > 0 ')
#         x = input('step 2: press any key to continue... ')
        
        
        no_match = df_media_no_match.join_concept.to_list()
#         print(f'{i}: search_media: {search_media}, no_match = {no_match}')
        
        for i, unmatched in enumerate(no_match):
#             print(f'{i}: unmatched = {unmatched}, remove_con = {remove_con}')
#             x = input('step 3: press any key to continue... ')
#             if cancel_repeat == True:
#                 break
            
            if i > 0:
                if unmatched == remove_this_con:
#                     print(f'already removed {remove_con} from  {search_media}')
#                     x = input(f'step 4: it worked! Continue? (press x)')
#                     cancel_repeat = True
                    break
            
            pos = False
            neg = False
            new_concept = None
            pop_index = None
            
            no_match_index = df_subjects.index(unmatched)
#             print(f'step 5: {i+1}: unmatched concept is: "{unmatched}" and its index is "{no_match_index}" out of {len(df_subjects)-1}')
            
            if no_match_index > 0:
                try_match_neg = df_subjects[no_match_index-1]+', '+df_subjects[no_match_index]
                concepts_found_pos = concepts.loc[concepts['join_concept'] == try_match_neg]
                if len(concepts_found_pos) > 0:
                    pos = True
#                     print(f'pos is {pos}')
            if no_match_index < len(df_subjects)-1:
                try_match_pos = df_subjects[no_match_index]+', '+df_subjects[no_match_index+1]
                concepts_found_neg = concepts.loc[concepts['join_concept'] == try_match_pos]
                if len(concepts_found_neg) > 0:
                    neg = True
#                     print(f'neg is {neg}')
#             print(f'try_match_neg = "{try_match_neg}"\ntry_match_pos = "{try_match_pos}"')
            
            try:
                if len(concepts_found_pos) == 1:
                    new_concept = concepts_found_pos.join_concept.to_list()[0]
                    
                        
            except (NameError):
                pass
            try:
                if len(concepts_found_neg) == 1:
                    new_concept = concepts_found_neg.join_concept.to_list()[0]
            except (NameError):
                pass
            try:
                if (len(concepts_found_pos) > 0) & (len(concepts_found_neg) > 0):
                    print('too many matches!')
                    pass
            except (NameError):
                pass
            try:
                if (len(concepts_found_pos) == 0) & (len(concepts_found_neg) == 0):
                    unmatched_df = df[(df['media'] == search_media) 
                                      & (df['join_concept'] == unmatched)][narrow_list]
                    add_to_not_found(unmatched_df)
            except (NameError):
                pass
            
            remove_con = df_subjects[no_match_index]
            remove_this_con_index = 0
            
            if pos:
                remove_this_con_index = no_match_index-1
            if neg:
                remove_this_con_index = no_match_index+1

            if pos | neg:
                remove_this_con = df_subjects[remove_this_con_index]

                df = df.drop(
                    df[((df['media'] == search_media) & 
                                      (df['join_concept'] == remove_this_con)
                                     )].index
                )
            
    #     determine the location in the index of the concept to be removed
                pop_index = df.loc[(df['media'] == search_media) & (df['join_concept'] == remove_con)].index
    #     remove the concept found at that index
                df.at[pop_index[0], 'join_concept'] = new_concept
                df.at[pop_index[0], 'subjects'] = new_concept
                df.at[pop_index[0], 'SUBJECT_ID_x'] = concepts.at[concepts.loc[concepts['join_concept'] == new_concept].index[0], 'SUBJECT_ID_x']
                df.at[pop_index[0], 'TERM_ID'] = concepts.at[concepts.loc[concepts['join_concept'] == new_concept].index[0], 'TERM_ID']

            print(f'new_concept = {new_concept}')
    

7500 of 7501: search_media = Arquin_069_0217.jpg


## Review Unmatched Results:

In [381]:
# read in the unmatched results
post_df_meda_no_match = df[df['SUBJECT_ID_x'].isna()]

In [389]:
unmatched_concepts_by_color = post_df_meda_no_match.groupby(['subjects'])['media'].agg('count').reset_index()

display(unmatched_concepts_by_color.sort_values('media', ascending=False))

,subjects,media
43,columns,208
145,trees,186
111,portal,92
120,sculptures,65
148,universities (buildings),62
...,...,...
65,fruit (plant components),1
62,fence,1
60,equilateral arche,1
56,domes,1


## A Step Through, as an Example

In [191]:
df = return_box(box_list[6])

In [192]:
search_media = 'B-1_Arquin_007_0257.jpg'

In [193]:
df[df['media'] == search_media].index

Int64Index([406], dtype='int64')

In [194]:
df = split_df(df)

In [195]:
# merge the df with the AAT Metadata 
df = pd.merge(df, concepts, how="left", on='join_concept')

In [196]:
# get the boxid
df['boxid_temp'] = df['media'].str.extract(r"(_0\d+_)")
df = df[df['media'].notna()]
df['boxid_temp'] = df['boxid_temp'].str.replace("_","")
df['boxid_temp'] = pd.to_numeric(df['boxid_temp'], errors='coerce')
df = df.dropna(subset=['boxid_temp'])
df['boxid_temp'] = df['boxid_temp'].astype('int')
df['boxid_temp'] = df['boxid_temp'].astype('str')
df['BoxId'] = 'Box '+df['boxid_temp']
df.drop('boxid_temp', inplace=True, axis=1)

In [197]:
# add the colors in 
color_match_df = pd.read_csv('csv_files/color_match', sep='\t')
df = pd.merge(df, color_match_df, on='BoxId')

In [198]:
# get all the unique media IDs where media is not NA 
df_medias = df.media.dropna().unique()
df_medias

array(['A-5_Arquin_007_0351.jpg', 'A-5_Arquin_007_0352.jpg',
       'A-5_Arquin_007_0353.jpg', 'A-5_Arquin_007_0354.jpg',
       'A-5_Arquin_007_0355.jpg', 'A-5_Arquin_007_0356.jpg',
       'A-5_Arquin_007_0357.jpg', 'A-5_Arquin_007_0358.jpg',
       'A-5_Arquin_007_0360.jpg', 'A-5_Arquin_007_0361.jpg',
       'A-5_Arquin_007_0362.jpg', 'A-5_Arquin_007_0363.jpg',
       'A-5_Arquin_007_0364.jpg', 'A-5_Arquin_007_0365.jpg',
       'A-5_Arquin_007_0366.jpg', 'A-5_Arquin_007_0367.jpg',
       'A-5_Arquin_007_0368.jpg', 'A-5_Arquin_007_0369.jpg',
       'A-5_Arquin_007_0370.jpg', 'A-5_Arquin_007_0371.jpg',
       'A-5_Arquin_007_0372.jpg', 'A-5_Arquin_007_0373.jpg',
       'A-5_Arquin_007_0374.jpg', 'A-5_Arquin_007_0376.jpg',
       'A-5_Arquin_007_0377.jpg', 'A-5_Arquin_007_0378.jpg',
       'A-5_Arquin_007_0379.jpg', 'A-5_Arquin_007_0380.jpg',
       'A-5_Arquin_007_0381.jpg', 'A-5_Arquin_007_0382.jpg',
       'A-5_Arquin_007_0383.jpg', 'A-5_Arquin_007_0384.jpg',
       'A-5_Arquin_007_0

In [199]:
# get the index of the search_media term in the array
search_media_index = df_medias.tolist().index(search_media)
search_media_index

193

In [200]:
# get the list of all the subjects for the subset of the dataframe for the box
df_subjects = df[df['media'] == search_media].join_concept.to_list()
df_subjects

['hospitals (institutions',
 'health facility)',
 'buildings (structures)',
 'signs (declatory or advertising artifacts)',
 'anonymous people']

In [203]:
# get the list of unmatched subjects
df_meda_no_match = df[df['media'] == search_media].loc[df['SUBJECT_ID_x'].isna()]
df_meda_no_match[narrow_list]

,identifier,media,subject,title,subjects,SUBJECT_ID_x,TERM_ID,join_concept,BoxId,color
861,B-1 257,B-1_Arquin_007_0257.jpg,"hospitals (institutions, health facility), bui...",Municipal Department of Health,hospitals (institutions,NaN,NaN,hospitals (institutions,Box 7,blue
862,B-1 257,B-1_Arquin_007_0257.jpg,"hospitals (institutions, health facility), bui...",Municipal Department of Health,health facility),NaN,NaN,health facility),Box 7,blue


In [204]:
# for any unmatched terms, check to see if combining the terms yields a match
no_match = df_meda_no_match.join_concept.to_list()
no_match

['hospitals (institutions', 'health facility)']

In [205]:
# identify the index value of the ummatched item
for i, unmatched in enumerate(no_match):
    no_match_index = df_subjects.index(unmatched)
    print(f'{i+1}: unmatched concept is: "{unmatched}" and its index is "{no_match_index}" out of {len(df_subjects)-1}')

1: unmatched concept is: "hospitals (institutions" and its index is "0" out of 4
2: unmatched concept is: "health facility)" and its index is "1" out of 4


In [229]:
no_match_index = df_subjects.index(no_match[1])
no_match_index

1

In [230]:
try_match_pos = df_subjects[no_match_index]+', '+df_subjects[no_match_index+1]
try_match_pos

'health facility), buildings (structures)'

In [231]:
try_match_neg = None
try_match_pos = None
concepts_found_pos = []
concepts_found_neg = []
pos = False
neg = False

if no_match_index > 0:
    try_match_neg = df_subjects[no_match_index-1]+', '+df_subjects[no_match_index]
    concepts_found_pos = concepts.loc[concepts['join_concept'] == try_match_neg]
    pos = True
if no_match_index < len(df_subjects)-1:
    try_match_pos = df_subjects[no_match_index]+', '+df_subjects[no_match_index+1]
    concepts_found_neg = concepts.loc[concepts['join_concept'] == try_match_pos]
    neg = True
print(f'try_match_neg = "{try_match_neg}"\ntry_match_pos = "{try_match_pos}"')

try_match_neg = "hospitals (institutions, health facility)"
try_match_pos = "health facility), buildings (structures)"


In [232]:
neg

True

In [233]:
concepts_found_pos[['SUBJECT_ID_x', 'TERM_ID', 'use_concept', 'join_concept']]

,SUBJECT_ID_x,TERM_ID,use_concept,join_concept
55395,300343473,1000460680,"hospitals (institutions, health facility)","hospitals (institutions, health facility)"


In [234]:
try:
    concepts_found_neg[['SUBJECT_ID_x', 'TERM_ID', 'use_concept', 'join_concept']]
except (NameError):
    print(f'NameError: concepts_found_neg not defined')

In [235]:
concepts_found_neg[['SUBJECT_ID_x', 'TERM_ID', 'use_concept', 'join_concept']]

,SUBJECT_ID_x,TERM_ID,use_concept,join_concept


In [236]:

try:
    if len(concepts_found_pos) == 1:
        new_concept = concepts_found_pos.join_concept.to_list()[0]
except (NameError):
    print(f'positive, but NameError')
try:
    if len(concepts_found_neg) == 1:
        new_concept = concepts_found_neg.join_concept.to_list()[0]
except (NameError):
    print(f'negative, but NameError')
try:
    if (len(concepts_found_pos) > 0) & (len(concepts_found_neg) > 0):
        print('too many matches!')
except (NameError):
    pass

new_concept

'hospitals (institutions, health facility)'

In [237]:
# determine the remove concept
remove_con = df_subjects[no_match_index]
remove_con

'health facility)'

In [238]:
# before the drop
df[df['media'] == search_media][narrow_list]

,identifier,media,subject,title,subjects,SUBJECT_ID_x,TERM_ID,join_concept,BoxId,color
861,B-1 257,B-1_Arquin_007_0257.jpg,"hospitals (institutions, health facility), bui...",Municipal Department of Health,"hospitals (institutions, health facility)",300343473,1000460680,"hospitals (institutions, health facility)",Box 7,blue
863,B-1 257,B-1_Arquin_007_0257.jpg,"hospitals (institutions, health facility), bui...",Municipal Department of Health,buildings (structures),300004792,1000004792,buildings (structures),Box 7,blue
864,B-1 257,B-1_Arquin_007_0257.jpg,"hospitals (institutions, health facility), bui...",Municipal Department of Health,signs (declatory or advertising artifacts),300123013,1000123013,signs (declatory or advertising artifacts),Box 7,blue
865,B-1 257,B-1_Arquin_007_0257.jpg,"hospitals (institutions, health facility), bui...",Municipal Department of Health,anonymous people,300435316,1000809718,anonymous people,Box 7,blue


In [239]:
remove_this_con_index = 0
if pos:
    remove_this_con_index = no_match_index-1
if neg:
    remove_this_con_index = no_match_index+1

remove_this_con = df_subjects[remove_this_con_index]

df = df.drop(
    df[((df['media'] == search_media) & 
                      (df['join_concept'] == remove_this_con)
                     )].index
)

In [240]:
# after the drop

df[df['media'] == search_media][narrow_list]

,identifier,media,subject,title,subjects,SUBJECT_ID_x,TERM_ID,join_concept,BoxId,color
861,B-1 257,B-1_Arquin_007_0257.jpg,"hospitals (institutions, health facility), bui...",Municipal Department of Health,"hospitals (institutions, health facility)",300343473,1000460680,"hospitals (institutions, health facility)",Box 7,blue
864,B-1 257,B-1_Arquin_007_0257.jpg,"hospitals (institutions, health facility), bui...",Municipal Department of Health,signs (declatory or advertising artifacts),300123013,1000123013,signs (declatory or advertising artifacts),Box 7,blue
865,B-1 257,B-1_Arquin_007_0257.jpg,"hospitals (institutions, health facility), bui...",Municipal Department of Health,anonymous people,300435316,1000809718,anonymous people,Box 7,blue


In [241]:
# replace the no_match value with the combined_concept

pop_index = df.loc[(df['media'] == search_media) & (df['join_concept'] == remove_con)].index
pop_index

Int64Index([], dtype='int64')

In [242]:
df.at[pop_index[0], 'join_concept'] = new_concept
df.at[pop_index[0], 'subjects'] = new_concept
df.at[pop_index[0], 'SUBJECT_ID_x'] = concepts.at[concepts.loc[concepts['join_concept'] == new_concept].index[0], 'SUBJECT_ID_x']
df.at[pop_index[0], 'TERM_ID'] = concepts.at[concepts.loc[concepts['join_concept'] == new_concept].index[0], 'TERM_ID']

IndexError: index 0 is out of bounds for axis 0 with size 0

In [228]:
df.loc[df['media']== search_media][narrow_list]

,identifier,media,subject,title,subjects,SUBJECT_ID_x,TERM_ID,join_concept,BoxId,color
861,B-1 257,B-1_Arquin_007_0257.jpg,"hospitals (institutions, health facility), bui...",Municipal Department of Health,"hospitals (institutions, health facility)",300343473,1000460680,"hospitals (institutions, health facility)",Box 7,blue
863,B-1 257,B-1_Arquin_007_0257.jpg,"hospitals (institutions, health facility), bui...",Municipal Department of Health,buildings (structures),300004792,1000004792,buildings (structures),Box 7,blue
864,B-1 257,B-1_Arquin_007_0257.jpg,"hospitals (institutions, health facility), bui...",Municipal Department of Health,signs (declatory or advertising artifacts),300123013,1000123013,signs (declatory or advertising artifacts),Box 7,blue
865,B-1 257,B-1_Arquin_007_0257.jpg,"hospitals (institutions, health facility), bui...",Municipal Department of Health,anonymous people,300435316,1000809718,anonymous people,Box 7,blue
